In [ ]:
pip install ultralytics

In [3]:
pip install yolov5

  Obtaining dependency information for yolov5 from https://files.pythonhosted.org/packages/1a/8d/ce2c22c4774fb5389d46921bea02f0efb8a409ec58f722949e96a62ef5c3/yolov5-7.0.13-py37.py38.py39.py310-none-any.whl.metadata
  Obtaining dependency information for ultralytics>=8.0.100 from https://files.pythonhosted.org/packages/95/db/b1f500f93a9bdf3e07e0c6e7bad78b152270c9d18926ab8ef50e8545d353/ultralytics-8.0.228-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for sahi>=0.11.10 from https://files.pythonhosted.org/packages/0b/da/23f351eb3360e58762f0d1ab2dc8521610cefb9e30e246eb715cbe337a38/sahi-0.11.15-py3-none-any.whl.metadata
  Obtaining dependency information for roboflow>=0.2.29 from https://files.pythonhosted.org/packages/95/da/72fda7a49ec81e369654eb6cfa733ec6e1ee87a7d1ed6a154c4cf6bf9c1d/roboflow-1.1.12-py3-none-any.whl.metadata
  Obtaining dependency 

In [5]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_boxes
import random

image_folder = "/kaggle/input/hateful-meme-challenge/hateful_memes/img"
weights_path = "/kaggle/input/yolo-5/yolov5s.pt"
model = attempt_load(weights_path)
model.eval()

object_count = {}
results_list = []
used_images_list = []
category_count_dict = {}

to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:3, :, :]),
])

all_images = os.listdir(image_folder)
subset_size = int(len(all_images) * 0.25)
random_subset = random.sample(all_images, subset_size)

for idx, filename in tqdm(enumerate(random_subset, start=1)):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(image_folder, filename)
        used_images_list.append({"Sr. No": idx, "Image": filename})
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = img.resize((640, 640))
        img_tensor = to_tensor(img)
        img_tensor = img_tensor.unsqueeze(0)

        results = model(img_tensor)
        results_np = non_max_suppression(results, conf_thres=0.5, iou_thres=0.5)[0]

        if results_np is not None:
            results_np[:, :4] = scale_boxes(img.size, results_np[:, :4], img.size).round()

            for detection in results_np:
                class_id = int(detection[5])
                confidence = detection[4]

                object_class = model.names[class_id]
                object_count[object_class] = object_count.get(object_class, 0) + 1

                if filename not in category_count_dict:
                    category_count_dict[filename] = {}
                category_count_dict[filename][object_class] = category_count_dict[filename].get(object_class, 0) + 1

                results_list.append({
                    "Sr. No": idx,
                    "Image": filename,
                    "Object_Class": object_class,
                    "Confidence": confidence
                })

used_images_df = pd.DataFrame(used_images_list)
results_df = pd.DataFrame(results_list)

used_images_df.to_csv("/kaggle/working/used_images_list.csv", index=False)
results_df.to_csv("/kaggle/working/object_detection_results.csv", index=False)

object_frequency_df = pd.DataFrame(list(object_count.items()), columns=["Object_Class", "Frequency"])
object_frequency_df.to_csv("/kaggle/working/object_frequency_distribution.csv", index=False)

category_count_results_df = pd.DataFrame(category_count_dict).T.reset_index()
category_count_results_df.rename(columns={"index": "Image name"}, inplace=True)
category_count_results_df.index.name = "Sr no"
category_count_results_df.to_csv("/kaggle/working/category_count_results.csv")

Fusing layers... 
YOLOv5s summary: 270 layers, 7235389 parameters, 0 gradients
3035it [10:47,  4.68it/s]
